In [84]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [90]:
eth_data = pd.read_csv('eth_daily_main.csv')

In [92]:
# Convert gold_price column to floats
for i in range(0,len(eth_data)):
    eth_data.at[i, 'gold_price'] = eth_data.at[i, 'gold_price'].replace(',', '')

In [93]:
eth_data

,Unnamed: 0,date,open,high,low,close,Volume USD,Volume ETH,pg_change_tomorrow,date_m,fed_rate,snp_close,rsi,cag,cal,ema,gold_price
0,0.0,2016-03-09 00:00:00,10.300000,11.20,10.10,11.200000,4.484300e+02,41.170000,4.910714,2016-03,0.36,1989.26,NaN,NaN,NaN,NaN,1303.40
1,1.0,2016-03-10 00:00:00,11.200000,11.85,11.07,11.750000,1.209000e+01,1.030000,1.702128,2016-03,0.36,1989.57,NaN,NaN,NaN,NaN,1320.10
2,2.0,2016-03-11 00:00:00,11.750000,11.95,11.75,11.950000,6.074200e+02,50.850000,8.117155,2016-03,0.36,2022.19,NaN,NaN,NaN,NaN,1308.60
3,3.0,2016-03-12 00:00:00,11.950000,13.45,11.95,12.920000,3.057520e+03,237.780000,16.640867,2016-03,0.36,2022.19,NaN,NaN,NaN,NaN,1308.60
4,4.0,2016-03-13 00:00:00,12.920000,15.07,12.92,15.070000,5.243670e+03,369.640000,-17.053749,2016-03,0.36,2022.19,NaN,NaN,NaN,NaN,1308.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2122.0,2021-12-30 00:00:00,3629.500000,3768.00,3586.30,3710.000000,8.040992e+07,21673.833371,-0.857143,2021-12,0.08,4778.73,36.595997,1.105018,1.634432,3924.921811,1814.10
2123,2123.0,2021-12-31 00:00:00,3710.600000,3816.80,3622.00,3678.200000,1.034630e+08,28128.693285,2.416943,2021-12,0.08,4766.18,40.337215,1.026088,1.578912,3892.025570,1828.60
2124,2124.0,2022-01-01 00:00:00,3677.600000,3777.90,3675.00,3767.100000,6.339609e+07,16828.885266,1.685647,2022-01,0.08,4766.18,39.389182,1.125435,1.466132,3875.368827,1828.60
2125,2125.0,2022-01-02 00:00:00,3767.171482,3852.50,3717.60,3830.600000,4.829641e+07,12608.053668,-0.459032,2022-01,0.08,4766.18,43.426808,1.165450,1.361409,3869.399650,1828.60


In [94]:
# removing rows with empty RSIs and EMAs
# Dropping irrelevant columns
model_data = eth_data.drop(eth_data.index[0:14])
model_data.drop(['Unnamed: 0', 'date_m', 'date', 'cag', 'cal'], axis=1, inplace=True)
model_data = model_data.rename(columns={'Volume USD': 'vol_usd', 'Volume ETH':'vol_eth'})
model_data.reset_index(drop=True, inplace=True)

In [95]:
# Putting percentage changes into categories for classification
temp_cat_list = []
for i in range(0, len(model_data)):
    change = model_data.at[i,'pg_change_tomorrow']
    print(change)
    if change >= 2 and change < 4:
        temp_cat_list.append('A')
    elif change >= 4 and change < 10:
        temp_cat_list.append('B')
    elif change >= 10:
        temp_cat_list.append('C')
    elif change <= -2 and change > -4:
        temp_cat_list.append('D')
    elif change <= -4 and change > -10:
        temp_cat_list.append('E')
    elif change <= -10:
        temp_cat_list.append('F')
    else:
        temp_cat_list.append('N')

model_data['future_change_ctgry'] = temp_cat_list

-9.895414320193085
-4.732142857142851
3.09278350515464
-5.454545454545452
11.730769230769221
1.4629948364888117
0.5089058524173071
-3.713080168776367
1.840490797546004
-0.08605851979345773
1.205857019810513
-5.106382978723404
-6.726457399103139
3.4615384615384563
-5.947955390334577
-4.05138339920947
-5.767250257466534
-4.295081967213123
-0.8792965627497997
-13.294930875576036
7.49402072814244
4.5117428924598295
-1.5966883500887024
2.656250000000001
10.701323030090151
-5.097831835007944
-2.8864370890448967
-1.147578609134742
-5.85094032969584
-1.738594327990136
6.161375329401449
-4.290780141843967
-6.891441274546146
-0.5305743467303361
3.987198293105741
-6.001538856116942
2.5238744884038145
18.057218895542256
-1.352569882777268
14.259597806215712
-6.570000000000001
1.2843840308252084
3.4555637747014685
-4.484167517875366
0.0
1.5934124692546157
-1.9157894736842145
0.6868426701008807
5.371988914943524
1.5577584462876606
5.577689243027893
-4.150943396226411
-1.9094488188976289
11.278346377

In [96]:
model_data.drop(model_data.index[2112], inplace=True)
model_data['gold_price'] = model_data['gold_price'].astype(float)
model_data

,open,high,low,close,vol_usd,vol_eth,pg_change_tomorrow,fed_rate,snp_close,rsi,ema,gold_price,future_change_ctgry
0,11.370000,12.43,11.20,12.43,9.222684e+04,7721.470000,-9.895414,0.36,2036.71,58.074007,11.989286,1274.0,E
1,12.430000,12.47,10.67,11.20,1.651251e+05,14512.640000,-4.732143,0.36,2035.94,58.074007,11.884048,1272.4,E
2,11.320000,11.35,10.52,10.67,2.706509e+05,24977.640000,3.092784,0.36,2035.94,52.846274,11.722175,1216.7,A
3,10.670000,11.20,10.58,11.00,5.034526e+04,4606.570000,-5.454545,0.36,2035.94,50.504885,11.625885,1216.7,E
4,10.930000,10.98,10.17,10.40,1.672745e+05,15947.080000,11.730769,0.36,2035.94,52.001677,11.462433,1216.7,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,3794.600000,3828.60,3606.00,3627.90,1.138826e+08,31390.790791,2.263017,0.08,4793.06,41.223554,3957.986705,1805.8,A
2108,3629.500000,3768.00,3586.30,3710.00,8.040992e+07,21673.833371,-0.857143,0.08,4778.73,36.595997,3924.921811,1814.1,N
2109,3710.600000,3816.80,3622.00,3678.20,1.034630e+08,28128.693285,2.416943,0.08,4766.18,40.337215,3892.025570,1828.6,A
2110,3677.600000,3777.90,3675.00,3767.10,6.339609e+07,16828.885266,1.685647,0.08,4766.18,39.389182,3875.368827,1828.6,N


In [97]:
X_data_rf = model_data.drop(['pg_change_tomorrow', 'future_change_ctgry'], axis=1)
Y_data_rf = model_data['future_change_ctgry'] 

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


X_train,X_test,Y_train,Y_test=train_test_split(X_data_rf,Y_data_rf,test_size=(0.3))

In [126]:
model_rf = RandomForestClassifier(random_state=1, criterion='entropy', max_depth=1)
model_rf.fit(X_train, Y_train)
print("vvvvvv RANDOM FOREST CLASSIFIER vvvvvv",model_rf.score(X_test, Y_test))
preds_rf = model_rf.predict(X_test)
print(classification_report(Y_test, preds_rf))

vvvvvv RANDOM FOREST CLASSIFIER vvvvvv 0.39747634069400634
              precision    recall  f1-score   support

           A       0.00      0.00      0.00        73
           B       0.00      0.00      0.00        89
           C       0.00      0.00      0.00        28
           D       0.00      0.00      0.00        90
           E       0.00      0.00      0.00        85
           F       0.00      0.00      0.00        17
           N       0.40      1.00      0.57       252

    accuracy                           0.40       634
   macro avg       0.06      0.14      0.08       634
weighted avg       0.16      0.40      0.23       634



C:\Users\sseba\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [130]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
X_minmax = minmax.fit_transform(X_train)
Xtest_minmax = minmax.fit_transform(X_test)

model_rfmm = RandomForestClassifier(random_state=1, criterion='entropy', max_depth=1)
model_rfmm.fit(X_minmax, Y_train)
print("vvvvvv RANDOM FOREST CLASSIFIER vvvvvv",model_rfmm.score(Xtest_minmax, Y_test))
preds_rf = model_rfmm.predict(Xtest_minmax)
print(classification_report(Y_test, preds_rf))

vvvvvv RANDOM FOREST CLASSIFIER vvvvvv 0.39747634069400634
              precision    recall  f1-score   support

           A       0.00      0.00      0.00        73
           B       0.00      0.00      0.00        89
           C       0.00      0.00      0.00        28
           D       0.00      0.00      0.00        90
           E       0.00      0.00      0.00        85
           F       0.00      0.00      0.00        17
           N       0.40      1.00      0.57       252

    accuracy                           0.40       634
   macro avg       0.06      0.14      0.08       634
weighted avg       0.16      0.40      0.23       634



C:\Users\sseba\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pred = model_rf.predict(np.array( [X_train,])